# Transformer (tylko decoder). 5 featerow - mag, x, y, depth, diff_days 

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

2023-11-21 01:44:08.223487: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-21 01:44:08.223575: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-21 01:44:08.227211: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-21 01:44:08.542401: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-21 01:44:11.521547: W tensorflow/compiler/

In [2]:
ds_train = tf.data.Dataset.load("../data/ds_final_train_old/")
ds_val = tf.data.Dataset.load("../data/ds_final_val_old/")
ds_test = tf.data.Dataset.load("../data/ds_final_test_old/")

2023-11-21 01:44:17.385023: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-21 01:44:17.776622: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-21 01:44:17.776724: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-21 01:44:17.782820: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-21 01:44:17.782916: I tensorflow/compile

In [3]:
BATCH_SIZE = 1024
BLOCK_SIZE = 64

In [4]:
def transform_element(features, context, labels):
    labels = tf.cast(labels, tf.int32)
    labels = tf.one_hot(labels, depth=2)
    return (features[:,:-2], features[:,-2], features[:,-1]), labels[0]

In [5]:
ds_train = ds_train.map(transform_element)
ds_val = ds_val.map(transform_element)
ds_test = ds_test.map(transform_element)

In [6]:
for x, y in ds_train.take(1):
    print(x[0].shape)
    print(x[1].shape)
    print(x[2].shape)
    print(y.shape)

(64, 6)
(64,)
(64,)
(2,)


In [7]:
ds_train = ds_train.batch(BATCH_SIZE).shuffle(1000).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
class MyModel(tf.keras.Model):
    def __init__(self, n_embed):
        super(MyModel, self).__init__()
        self.embed_dd = tf.keras.layers.Embedding(100, n_embed // 8)
        self.embed_plate = tf.keras.layers.Embedding(100, n_embed // 8)
        self.lstm_embed = tf.keras.layers.LSTM(n_embed // 8 * 6, return_sequences=True)
        self.lstm1 = tf.keras.layers.LSTM(n_embed, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(n_embed)
        self.dense1 = tf.keras.layers.Dense(n_embed, activation='relu')
        self.dense2 = tf.keras.layers.Dense(2)
        self.conc = tf.keras.layers.Concatenate(axis=-1)

    def call(self, inputs):
        cont, plate, dd = inputs
        x1 = self.embed_dd(dd)
        x2 = self.embed_plate(plate)
        x3 = self.lstm_embed(cont)
        x = self.conc([x1, x2, x3])
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [9]:
model = MyModel(128)

In [10]:
model.build(input_shape=[(None, 64, 6), (None, 64), (None, 64)])
model.count_params()

322690

In [11]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [12]:
history = model.fit(ds_train, epochs=50, validation_data=ds_val)

Epoch 1/50


2023-11-21 01:45:02.386939: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 140 of 1000
2023-11-21 01:45:12.411419: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 299 of 1000
2023-11-21 01:45:22.419741: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 466 of 1000
2023-11-21 01:45:42.438432: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 793 of 1000
2023-11-21 01:45:52.439662: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 964 of 1000
2023-11-21 01:45:54.551193: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2023-11-21 01:45:56.451064: I tensorflow/

3227/3227 [==============================] - 429s 110ms/step - loss: 0.1713 - accuracy: 0.9446 - f1_score: 0.6280 - val_loss: 0.2066 - val_accuracy: 0.9424 - val_f1_score: 0.5907
Epoch 2/50


2023-11-21 01:52:02.714886: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 147 of 1000
2023-11-21 01:52:12.715113: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 300 of 1000
2023-11-21 01:52:32.756288: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 612 of 1000
2023-11-21 01:52:42.757915: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 781 of 1000
2023-11-21 01:52:55.806730: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


3227/3227 [==============================] - 481s 129ms/step - loss: 0.1600 - accuracy: 0.9481 - f1_score: 0.6718 - val_loss: 0.2160 - val_accuracy: 0.9422 - val_f1_score: 0.5816
Epoch 3/50


...

2023-11-21 09:29:26.691667: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 174 of 1000
2023-11-21 09:29:46.643706: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 491 of 1000
2023-11-21 09:29:56.648925: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 626 of 1000
2023-11-21 09:30:06.670922: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 786 of 1000
2023-11-21 09:30:16.677985: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 934 of 1000
2023-11-21 09:30:21.445894: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


3227/3227 [==============================] - 1015s 295ms/step - loss: 0.0791 - accuracy: 0.9733 - f1_score: 0.8658 - val_loss: 0.2871 - val_accuracy: 0.9336 - val_f1_score: 0.6494
Epoch 31/50


2023-11-21 09:46:22.083453: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 152 of 1000
2023-11-21 09:46:32.102645: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 294 of 1000
2023-11-21 09:46:52.083067: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 597 of 1000
2023-11-21 09:47:02.098794: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 743 of 1000
2023-11-21 09:47:12.118993: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 883 of 1000
2023-11-21 09:47:20.974068: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


2582/3227 [=======================>......] - ETA: 2:48 - loss: 0.0813 - accuracy: 0.9723 - f1_score: 0.8663

KeyboardInterrupt: 

In [ ]:
model.evaluate(ds_test)

105/105 [==============================] - 9s 82ms/step - loss: 0.0834 - accuracy: 0.9783 - f1_score: 0.6870


[0.0833962932229042,
 0.9783322215080261,
 array([0.9889718 , 0.38498402], dtype=float32)]

In [ ]:
model.save("../models/model_v2.keras")

In [ ]:
# save history
history = pd.DataFrame.from_dict(history.history)
history.to_csv("../models/history_model_v2.csv", index=False)